In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from flask import Flask,request,jsonify
import json
from pymongo import MongoClient
import time
import threading

In [9]:

thread6 = threading.Thread(target = crawler,args=(195000,196000))
thread7 = threading.Thread(target = crawler,args=(196000,197000))
thread8 = threading.Thread(target = crawler,args=(197000,198000))
thread9 = threading.Thread(target = crawler,args=(198000,199000))
thread10 = threading.Thread(target = crawler,args=(199000,200000))

thread6.start()
thread7.start()
thread8.start()
thread9.start()
thread10.start()

thread6.join()
thread7.join()
thread8.join()
thread9.join()
thread10.join()

print("all done")

recipe 199000 saved seccessfully.
recipe 199001 not found
recipe 199002 not found
recipe 199003 not found
recipe 196000 saved seccessfully.
recipe 199004 not found
recipe 199005 not found
recipe 197000 saved seccessfully.
recipe 196001 saved seccessfully.
recipe 196002 not found
recipe 199006 saved seccessfully.
recipe 195000 saved seccessfully.
recipe 198000 saved seccessfully.
recipe 195001 saved seccessfully.
recipe 195002 not found
recipe 197001 saved seccessfully.
recipe 196003 saved seccessfully.
recipe 199007 saved seccessfully.
recipe 198001 saved seccessfully.
recipe 198002 not found
recipe 198003 not found
recipe 197002 saved seccessfully.
recipe 195003 saved seccessfully.
recipe 199008 saved seccessfully.
recipe 196004 saved seccessfully.
recipe 197003 saved seccessfully.
recipe 198004 saved seccessfully.
recipe 196005 saved seccessfully.
recipe 199009 saved seccessfully.
recipe 197004 saved seccessfully.
recipe 196006 saved seccessfully.
recipe 195004 saved seccessfully.
re

recipe 199055 saved seccessfully.
recipe 195042 not found
recipe 199056 not found
recipe 195043 not found
recipe 199057 not found
recipe 196057 saved seccessfully.
recipe 197045 saved seccessfully.
recipe 196058 saved seccessfully.
recipe 195044 saved seccessfully.
recipe 198078 saved seccessfully.
recipe 198079 not found
recipe 199058 saved seccessfully.
recipe 198080 saved seccessfully.
recipe 195045 saved seccessfully.
recipe 197046 saved seccessfully.
recipe 195046 saved seccessfully.
recipe 196059 saved seccessfully.
recipe 195047 not found
recipe 10 not found
recipe 198081 saved seccessfully.
recipe 195048 saved seccessfully.
recipe 199060 saved seccessfully.
recipe 195049 not found
recipe 197047 saved seccessfully.
recipe 199061 not found
recipe 196060 saved seccessfully.
recipe 199062 not found
recipe 197048 not found
recipe 199063 not found
recipe 195050 saved seccessfully.
recipe 196061 not found
recipe 197049 not found
recipe 195051 not found
recipe 198082 saved seccessfully

recipe 198126 saved seccessfully.
recipe 199111 saved seccessfully.
recipe 197096 saved seccessfully.
recipe 197097 not found
recipe 196119 saved seccessfully.
recipe 195098 saved seccessfully.
recipe 196120 not found
recipe 199112 saved seccessfully.
recipe 197098 saved seccessfully.
recipe 198127 saved seccessfully.
recipe 199113 not found
recipe 198128 not found
recipe 196121 saved seccessfully.
recipe 195099 saved seccessfully.
recipe 197099 saved seccessfully.
recipe 199114 saved seccessfully.
recipe 199115 not found
recipe 195100 saved seccessfully.
recipe 199116 not found
recipe 196122 saved seccessfully.
recipe 196123 not found
recipe 197100 saved seccessfully.
recipe 198129 saved seccessfully.
recipe 199117 saved seccessfully.
recipe 199118 not found
recipe 198130 saved seccessfully.
recipe 195101 saved seccessfully.
recipe 196124 saved seccessfully.
recipe 197101 saved seccessfully.
recipe 196125 not found
recipe 196126 not found
recipe 199119 saved seccessfully.
recipe 19612

recipe 198179 saved seccessfully.
recipe 198180 not found
recipe 199161 saved seccessfully.
recipe 198181 not found
recipe 198182 not found
recipe 198183 not found
recipe 199162 saved seccessfully.
recipe 195157 saved seccessfully.
recipe 195158 not found
recipe 196184 saved seccessfully.
recipe 197147 saved seccessfully.
recipe 198184 saved seccessfully.
recipe 195159 not found
recipe 199163 saved seccessfully.
recipe 196185 not found
recipe 198185 not found
recipe 195160 not found
recipe 196186 not found
recipe 195161 not found
recipe 196187 not found
recipe 195162 not found
recipe 198186 saved seccessfully.
recipe 196188 saved seccessfully.
recipe 196189 not found
recipe 195163 saved seccessfully.
recipe 195164 not found
recipe 195165 not found
recipe 195166 not found
recipe 197148 saved seccessfully.
recipe 198187 saved seccessfully.
recipe 195167 not found
recipe 198188 not found
recipe 195168 not found
recipe 196190 saved seccessfully.
recipe 199164 saved seccessfully.
recipe 197

recipe 199215 saved seccessfully.
recipe 195213 not found
recipe 199216 not found
recipe 195214 not found
recipe 197200 saved seccessfully.
recipe 196242 saved seccessfully.recipe 195215 not found

recipe 4 not found
recipe 199217 saved seccessfully.
recipe 199218 not found
recipe 199219 not found
recipe 195216 saved seccessfully.
recipe 195217 not found
recipe 196243 saved seccessfully.
recipe 195218 not found
recipe 197201 saved seccessfully.
recipe 195219 not foundrecipe 197202 not found

recipe 198229 saved seccessfully.
recipe 198230 not found
recipe 196244 saved seccessfully.
recipe 199220 saved seccessfully.
recipe 196245 saved seccessfully.
recipe 195220 saved seccessfully.
recipe 197203 saved seccessfully.
recipe 195221 not found
recipe 197204 not foundrecipe 195222 not found

recipe 199221 saved seccessfully.
recipe 199222 not found
recipe 196246 saved seccessfully.
recipe 199223 not found
recipe 198231 saved seccessfully.
recipe 197205 saved seccessfully.
recipe 198232 not f

...

recipe 197712 saved seccessfully.
recipe 195799 saved seccessfully.
recipe 198864 saved seccessfully.
recipe 196748 saved seccessfully.
recipe 195800 not found
recipe 199754 saved seccessfully.
recipe 197713 saved seccessfully.
recipe 198865 saved seccessfully.
recipe 199755 saved seccessfully.
recipe 195801 saved seccessfully.
recipe 199756 not found
recipe 196749 saved seccessfully.
recipe 197714 saved seccessfully.
recipe 195802 saved seccessfully.
recipe 195803 not found
recipe 196750 saved seccessfully.
recipe 198866 saved seccessfully.
recipe 199757 saved seccessfully.
recipe 195804 not found
recipe 197715 saved seccessfully.
recipe 196751 not found
recipe 195805 not found
recipe 199758 not found
recipe 195806 not found
recipe 195807 not found
recipe 198867 saved seccessfully.
recipe 199759 saved seccessfully.
recipe 197716 saved seccessfully.
recipe 196752 saved seccessfully.
recipe 195808 saved seccessfully.
recipe 198868 saved seccessfully.
recipe 199760 saved seccessfully.
re

recipe 197755 saved seccessfully.
recipe 199811 not found
recipe 195855 not found
recipe 199812 not found
recipe 195856 not found
recipe 199813 not found
recipe 199814 not found
recipe 198924 saved seccessfully.
recipe 197756 saved seccessfully.
recipe 195857 saved seccessfully.
recipe 198925 saved seccessfully.
recipe 196800 saved seccessfully.
recipe 196801 not found
recipe 199815 saved seccessfully.
recipe 195858 saved seccessfully.
recipe 197757 saved seccessfully.
recipe 197758 not found
recipe 196802 saved seccessfully.
recipe 199816 saved seccessfully.
recipe 198926 saved seccessfully.
recipe 199817 not found
recipe 199818 not found
recipe 195859 saved seccessfully.
recipe 198927 saved seccessfully.
recipe 197759 saved seccessfully.
recipe 196803 saved seccessfully.
recipe 198928 not found
recipe 199819 saved seccessfully.
recipe 198929 not found
recipe 196804 not found
recipe 195860 saved seccessfully.
recipe 199820 saved seccessfully.
recipe 199821 not found
recipe 197760 save

recipe 195913 saved seccessfully.
recipe 196846 saved seccessfully.
recipe 197809 saved seccessfully.
recipe 196847 not found
recipe 198975 saved seccessfully.
recipe 196848 not found
recipe 196849 not found
recipe 196850 not found
recipe 198976 saved seccessfully.
recipe 198977 not found
recipe 195914 saved seccessfully.
recipe 197810 saved seccessfully.
recipe 196851 saved seccessfully.
recipe 198978 saved seccessfully.
recipe 195915 saved seccessfully.
recipe 197811 saved seccessfully.
recipe 199872 saved seccessfully.
recipe 199873 not found
recipe 196852 saved seccessfully.
recipe 198979 saved seccessfully.
recipe 199874 saved seccessfully.
recipe 195916 saved seccessfully.
recipe 195917 not found
recipe 196853 saved seccessfully.
recipe 198980 saved seccessfully.
recipe 199875 saved seccessfully.
recipe 199876 not found
recipe 198981 saved seccessfully.
recipe 195918 saved seccessfully.
recipe 197812 saved seccessfully.
recipe 195919 not found
recipe 197813 not found
recipe 19592

recipe 196911 saved seccessfully.
recipe 195981 not found
recipe 195982 not found
recipe 195983 not found
recipe 195984 not found
recipe 196912 saved seccessfully.
recipe 199929 saved seccessfully.
recipe 197866 saved seccessfully.
recipe 196913 saved seccessfully.
recipe 196914 not found
recipe 195985 saved seccessfully.
recipe 195986 not found
recipe 197867 saved seccessfully.
recipe 199930 saved seccessfully.
recipe 199931 not found
recipe 195987 saved seccessfully.
recipe 199932 not found
recipe 195988 not found
recipe 196915 saved seccessfully.
recipe 196916 saved seccessfully.
recipe 197868 saved seccessfully.
recipe 199933 saved seccessfully.
recipe 197869 saved seccessfully.
recipe 195989 saved seccessfully.
recipe 195990 not found
recipe 199934 saved seccessfully.
recipe 197870 saved seccessfully.
recipe 195991 saved seccessfully.
recipe 197871 saved seccessfully.
recipe 197872 not found
recipe 199935 saved seccessfully.
recipe 195992 saved seccessfully.
recipe 195993 not foun

recipe 197961 saved seccessfully.
recipe 197962 not found
recipe 197963 not found
recipe 197964 saved seccessfully.
recipe 197965 not found
recipe 197966 saved seccessfully.
recipe 197967 saved seccessfully.
recipe 197968 saved seccessfully.
recipe 197969 saved seccessfully.
recipe 197970 saved seccessfully.
recipe 197971 saved seccessfully.
recipe 197972 not found
recipe 197973 not found
recipe 197974 saved seccessfully.
recipe 197975 not found
recipe 197976 saved seccessfully.
recipe 197977 saved seccessfully.
recipe 197978 not found
recipe 197979 not found
recipe 197980 not found
recipe 197981 saved seccessfully.
recipe 197982 not found
recipe 197983 saved seccessfully.
recipe 197984 saved seccessfully.
recipe 197985 saved seccessfully.
recipe 197986 not found
recipe 197987 saved seccessfully.
recipe 197988 not found
recipe 197989 not found
recipe 197990 saved seccessfully.
recipe 197991 not found
recipe 197992 not found
recipe 197993 not found
recipe 197994 not found
recipe 197995 

In [8]:
# start from here
thread1 = threading.Thread(target = crawler,args=(190000,191000))
thread2 = threading.Thread(target = crawler,args=(191000,192000))
thread3 = threading.Thread(target = crawler,args=(192000,193000))
thread4 = threading.Thread(target = crawler,args=(193000,194000))
thread5 = threading.Thread(target = crawler,args=(194000,195000))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()


thread1.join()
thread2.join()
thread3.join()
thread4.join()
thread5.join()
print("all done")

recipe 191000 saved seccessfully.
recipe 193000 saved seccessfully.
recipe 190000 saved seccessfully.
recipe 192000 saved seccessfully.
recipe 190001 saved seccessfully.
recipe 194000 saved seccessfully.
recipe 190002 not found
recipe 190003 not found
recipe 193001 saved seccessfully.
recipe 190004 not found
recipe 191001 saved seccessfully.
recipe 190005 not found
recipe 192001 saved seccessfully.
recipe 190006 not found
recipe 192002 not found
recipe 190007 not found
recipe 191002 saved seccessfully.
recipe 192003 saved seccessfully.
recipe 193002 saved seccessfully.
recipe 192004 not found
recipe 192005 not found
recipe 191003 saved seccessfully.
recipe 193003 saved seccessfully.
recipe 193004 not found
recipe 190008 saved seccessfully.
recipe 190009 not found
recipe 192006 saved seccessfully.
recipe 190010 not found
recipe 193005 saved seccessfully.recipe 192007 not found

recipe 190011 not found
recipe 194001 saved seccessfully.
recipe 194002 not found
recipe 194003 not found
reci

recipe 191056 saved seccessfully.
recipe 190053 not found
recipe 192057 saved seccessfully.
recipe 191057 not found
recipe 190054 not found
recipe 191058 not found
recipe 194057 saved seccessfully.
recipe 194058 not found
recipe 190055 saved seccessfully.
recipe 191059 saved seccessfully.
recipe 193054 saved seccessfully.
recipe 191060 not found
recipe 192058 saved seccessfully.
recipe 191061 saved seccessfully.
recipe 192059 not found
recipe 191062 not found
recipe 193055 saved seccessfully.
recipe 192060 not found
recipe 191063 not found
recipe 193056 not found
recipe 190056 saved seccessfully.
recipe 194059 saved seccessfully.
recipe 194060 saved seccessfully.
recipe 193057 saved seccessfully.
recipe 191064 saved seccessfully.
recipe 192061 saved seccessfully.
recipe 194061 saved seccessfully.
recipe 193058 saved seccessfully.
recipe 190057 saved seccessfully.
recipe 192062 saved seccessfully.
recipe 193059 saved seccessfully.
recipe 190058 not found
recipe 191065 saved seccessfully

recipe 194111 not found
recipe 191123 not found
recipe 192109 saved seccessfully.
recipe 193111 saved seccessfully.
recipe 194112 saved seccessfully.
recipe 193112 saved seccessfully.
recipe 193113 not found
recipe 193114 not found
recipe 190097 saved seccessfully.
recipe 191124 saved seccessfully.
recipe 192110 saved seccessfully.
recipe 194113 saved seccessfully.
recipe 192111 not found
recipe 190098 saved seccessfully.
recipe 192112 not found
recipe 192113 not found
recipe 190099 saved seccessfully.
recipe 192114 not found
recipe 191125 saved seccessfully.
recipe 193115 saved seccessfully.
recipe 194114 saved seccessfully.
recipe 191126 not found
recipe 194115 not found
recipe 191127 not found
recipe 194116 not found
recipe 194117 not found
recipe 190100 saved seccessfully.
recipe 194118 saved seccessfully.
recipe 3 not found
recipe 193116 saved seccessfully.
recipe 192115 saved seccessfully.
recipe 190101 saved seccessfully.
recipe 191129 saved seccessfully.
recipe 191130 not found

recipe 194170 saved seccessfully.
recipe 191179 saved seccessfully.
recipe 191180 not found
recipe 192167 saved seccessfully.
recipe 194171 saved seccessfully.
recipe 190157 saved seccessfully.
recipe 191181 saved seccessfully.
recipe 194172 saved seccessfully.
recipe 191182 not found
recipe 190158 saved seccessfully.recipe 194173 not found

recipe 194174 not found
recipe 193158 saved seccessfully.
recipe 194175 not found
recipe 193159 not found
recipe 194176 not found
recipe 193160 not found
recipe 193161 not found
recipe 192168 saved seccessfully.
recipe 192169 not found
recipe 191183 saved seccessfully.
recipe 190159 saved seccessfully.
recipe 190160 not found
recipe 191184 saved seccessfully.
recipe 191185 not found
recipe 191186 not found
recipe 191187 not found
recipe 191188 not found
recipe 193162 saved seccessfully.
recipe 192170 saved seccessfully.
recipe 194177 saved seccessfully.
recipe 190161 saved seccessfully.
recipe 190162 not found
recipe 192171 saved seccessfully.
reci

recipe 190218 not found
recipe 190219 not found
recipe 193206 saved seccessfully.
recipe 8 not found
recipe 192217 saved seccessfully.
recipe 194222 saved seccessfully.
recipe 190220 saved seccessfully.
recipe 192218 saved seccessfully.
recipe 190221 not found
recipe 191242 saved seccessfully.
recipe 191243 not found
recipe 194223 saved seccessfully.
recipe 190222 saved seccessfully.
recipe 193207 saved seccessfully.
recipe 190223 not found
recipe 190224 not found
recipe 190225 not found
recipe 192219 saved seccessfully.
recipe 190226 saved seccessfully.
recipe 194224 saved seccessfully.
recipe 190227 not found
recipe 191244 saved seccessfully.
recipe 192220 saved seccessfully.
recipe 194225 saved seccessfully.
recipe 192221 not found
recipe 194226 not found
recipe 193208 saved seccessfully.
recipe 194227 not found
recipe 190228 saved seccessfully.
recipe 194228 not found
recipe 190229 not found
recipe 191245 saved seccessfully.
recipe 194229 not found
recipe 190230 not found
recipe 19

...

recipe 192779 not found
recipe 193767 not found
recipe 190806 saved seccessfully.
recipe 192780 not found
recipe 193768 not found
recipe 192781 not found
recipe 192782 not found
recipe 191782 saved seccessfully.
recipe 193769 saved seccessfully.
recipe 190807 saved seccessfully.
recipe 191783 not foundrecipe 194714 saved seccessfully.

recipe 193770 not found
recipe 191784 not found
recipe 193771 not found
recipe 193772 not found
recipe 194715 saved seccessfully.
recipe 193773 not found
recipe 191785 saved seccessfully.
recipe 192783 saved seccessfully.
recipe 192784 not found
recipe 192785 not found
recipe 190808 saved seccessfully.
recipe 194716 saved seccessfully.
recipe 191786 saved seccessfully.
recipe 190809 saved seccessfully.
recipe 191787 not found
recipe 193774 saved seccessfully.
recipe 192786 saved seccessfully.
recipe 190810 saved seccessfully.
recipe 194717 saved seccessfully.
recipe 192787 not found
recipe 192788 not found
recipe 190811 saved seccessfully.
recipe 192789 

recipe 191837 not found
recipe 193818 saved seccessfully.
recipe 193819 not found
recipe 192857 saved seccessfully.
recipe 193820 saved seccessfully.
recipe 192858 saved seccessfully.
recipe 193821 not found
recipe 193822 saved seccessfully.
recipe 194758 saved seccessfully.
recipe 193823 not found
recipe 194759 saved seccessfully.
recipe 190857 not found
recipe 193824 saved seccessfully.
recipe 192859 saved seccessfully.
recipe 193825 saved seccessfully.
recipe 191838 saved seccessfully.
recipe 192860 not found
recipe 191839 not found
recipe 192861 not found
recipe 190858 saved seccessfully.
recipe 194760 saved seccessfully.
recipe 193826 saved seccessfully.
recipe 194761 saved seccessfully.
recipe 192862 saved seccessfully.
recipe 192863 not found
recipe 193827 saved seccessfully.
recipe 194762 saved seccessfully.
recipe 194763 not found
recipe 194764 not found
recipe 190859 saved seccessfully.
recipe 191840 saved seccessfully.
recipe 193828 saved seccessfully.
recipe 191841 not foun

recipe 192898 not found
recipe 193875 not found
recipe 194847 not found
recipe 193876 not found
recipe 191881 saved seccessfully.
recipe 194848 not found
recipe 192899 saved seccessfully.
recipe 191882 saved seccessfully.
recipe 193877 saved seccessfully.
recipe 191883 not found
recipe 193878 not found
recipe 191884 not found
recipe 190911 saved seccessfully.
recipe 192900 saved seccessfully.
recipe 190912 not found
recipe 194849 saved seccessfully.
recipe 191885 saved seccessfully.
recipe 191886 not foundrecipe 190913 saved seccessfully.

recipe 194850 saved seccessfully.
recipe 190914 not found
recipe 191887 not found
recipe 194851 not found
recipe 192901 saved seccessfully.
recipe 193879 saved seccessfully.
recipe 194852 saved seccessfully.
recipe 191888 saved seccessfully.
recipe 193880 not found
recipe 194853 not found
recipe 190915 saved seccessfully.
recipe 193881 not found
recipe 193882 not found
recipe 191889 saved seccessfully.
recipe 194854 saved seccessfully.
recipe 193883 

recipe 193936 saved seccessfully.
recipe 190972 saved seccessfully.
recipe 194901 saved seccessfully.
recipe 194902 not found
recipe 194903 not found
recipe 9 not found
recipe 193937 saved seccessfully.
recipe 193938 not found
recipe 190973 saved seccessfully.
recipe 192942 saved seccessfully.
recipe 193939 not found
recipe 191938 saved seccessfully.
recipe 190974 not found
recipe 193940 not found
recipe 193941 not found
recipe 192943 saved seccessfully.
recipe 192944 not found
recipe 193942 saved seccessfully.
recipe 190975 saved seccessfully.
recipe 190976 not found
recipe 194904 saved seccessfully.
recipe 191939 saved seccessfully.
recipe 194905 not found
recipe 193943 saved seccessfully.
recipe 190977 saved seccessfully.
recipe 190978 not found
recipe 191940 saved seccessfully.
recipe 190979 not found
recipe 192945 saved seccessfully.
recipe 192946 not found
recipe 190980 saved seccessfully.
recipe 194906 saved seccessfully.
recipe 193944 saved seccessfully.
recipe 192947 not found

recipe 194966 saved seccessfully.
recipe 194967 not found
recipe 191993 saved seccessfully.
recipe 194968 saved seccessfully.
recipe 191994 saved seccessfully.
recipe 194969 saved seccessfully.
recipe 194970 saved seccessfully.
recipe 191995 saved seccessfully.
recipe 194971 saved seccessfully.
recipe 194972 not found
recipe 191996 saved seccessfully.
recipe 194973 saved seccessfully.
recipe 191997 not found
recipe 191998 saved seccessfully.
recipe 191999 saved seccessfully.
recipe 194974 saved seccessfully.
recipe 194975 saved seccessfully.
recipe 194976 not found
recipe 194977 saved seccessfully.
recipe 194978 saved seccessfully.
recipe 194979 not found
recipe 194980 saved seccessfully.
recipe 194981 saved seccessfully.
recipe 194982 saved seccessfully.
recipe 194983 saved seccessfully.
recipe 194984 not found
recipe 194985 saved seccessfully.
recipe 194986 saved seccessfully.
recipe 194987 not found
recipe 194988 saved seccessfully.
recipe 194989 not found
recipe 194990 saved secces

In [2]:
client = MongoClient('localhost', 27017)
db = client['recipe_db']
collection = db.recipe_collection

# num = 1000
def crawler(start,end):
    for i in range(start,end):
        try : 
            url = 'https://icook.tw/recipes/{}'.format(i)
            userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            headers = {
            'User-Agent':userAgent
            }
            ss = requests.session()
            response = ss.get(url,headers=headers)
            soup = BeautifulSoup(response.text,'html.parser')
            soup
            ingredientDict = {} 
            cooking_step = {}
            cooking_step_list = []

            dish_info = {}

            # dish_name
            dish_name = soup.select('h1[class="title"]')[0].text.strip()
            dish_info['DishName'] = dish_name

            # dishid
            dishid = i
            dish_info['dishid']= i

            # author
            authorname = soup.select('div[class="author-name"]')[0].text.strip()
            dish_info['author'] = authorname


            # author info
            try:
                author_info_list = soup.select('span[class="author-stat"]')
                author_recipe = author_info_list[0].select('span[class="stat-num"]')[0].text
                author_fans = author_info_list[1].select('span[class="stat-num"]')[0].text
                author_info = {'author_recipe':author_recipe,'author_fans':author_fans}
                dish_info['author_info'] = author_info
            except:
                pass

            # imageurl
            imageUrl = soup.select('div[class="recipe-cover"]')[0].select('img')[0]['src']
            dish_info['imageUrl'] = imageUrl
            resImg = requests.get(imageUrl, headers=headers)
            resContent = resImg.content  # 取出來的資料為二進制內容
            with open('./recipe_photo/{}.jpg'.format(dishid),'wb') as file:
                file.write(resContent)

            # description
            try:
                descriptiontext = soup.select('section[class="description"]')[0].select('p')[0].text
                dish_info['description'] = descriptiontext
            except:
                pass

            # portion
            try:
                portion = soup.select('div[class="servings"]')[0].select('span[class="num"]')[0].text
                dish_info['portion'] = portion
            except:
                pass

            # cooking time
            try:
                time = soup.select('div[class="info-content"]')[1].select('span[class="num"]')[0].text
                dish_info['cooking_time'] = time
            except:
                pass

            # ingredient
            ingredient_name_list = soup.select('div[class="ingredient-name"]')
            ingredient_unit_list = soup.select('div[class="ingredient-unit"]')

            for i in range(len(ingredient_name_list)): 
                ingredient_name = ingredient_name_list[i].text.strip()
                ingredient_unit = ingredient_unit_list[i].text
                ingredientDict[ingredient_name] = ingredient_unit

            dish_info['ingredient'] = ingredientDict

            #cooking steps
            steps = soup.select('ul[class="recipe-details-steps"]')[0].text.strip().split('\n')
            for item in steps:
                if len(item) > 1:
                    cooking_step_list.append(item)

            dish_info['cooking_step'] = cooking_step_list

            # tag 
            try:
                tagList = soup.select('ul[class="breadcrumb"]')
                if len(tagList) >=1:
                    tag = []
                    for item in tagList:
                        try:
                            tag.append(item.select('li[class="breadcrumb-item"]')[2].text.strip())
                        except:
                            pass
                        try:
                            tag.append(item.select('li[class="breadcrumb-item"]')[3].text.strip())
                        except:
                            pass

                    dish_info['tags'] = tag
            except:
                pass

            # likes
            try : 
                likesnum = soup.select('span[class="stat-left"]')[0].select('span[class="stat-content"]')[0].text
                dish_info['likes'] = likesnum
            except:
                pass

            # views
            views = soup.select('div[class="recipe-detail-meta-item"]')[0].text.strip()
            dish_info['views'] = views

    #         print(dish_info)
            recipe_id = collection.insert_one(dish_info).inserted_id
            print(f'recipe {dishid} saved seccessfully.')
        except:
            print(f'recipe {i} not found')

In [ ]:
13962